# > Tile World

# Installs

In [337]:
#Instalador de las librerias necesarias
#
#%pip install owlready2 agentpy seaborn numpy
#

In [338]:
# Model design
from owlready2 import *
import agentpy as ap
import numpy as np

# Visualization
import seaborn as sns

#Pathfinding
import math
import heapq

#Misc
from matplotlib import pyplot as plt
import IPython
import random

#Pathfinding Algorithm

In [339]:
def heuristic(a,b):
  #Distancia de Manhattan, resta el valor absoluto de la XY axtual con la XY destino
  # a[0] = x inicial
  # b[0] = x final
  # a[1] = y inicial
  # b[1] = y final
  return abs(a[0] - b[0]) + abs(a[1] - b[1])

"""
def a_star(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))

    # Mapeo del camino
    came_from = {}

    # Generar todas las posiciones posibles dentro del grid
    all_positions = [(x, y) for x in range(grid.shape[0]) for y in range(grid.shape[1])]

    # Inicializar g_score y f_score
    g_score = {node: float("inf") for node in all_positions}
    g_score[start] = 0

    f_score = {node: float("inf") for node in all_positions}
    f_score[start] = heuristic(start, goal)

    while open_list:
        current = heapq.heappop(open_list)[1]

        # Si se alcanza el objetivo, reconstruir y devolver el camino
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]  # Devolver el camino invertido

        # Explorar vecinos
        for neighbor in get_neighbors(grid, current):
            tentative_g_score = g_score[current] + 1  # Suponiendo un costo uniforme

            if tentative_g_score < g_score[neighbor]:
                # Este camino es mejor que el anterior, actualizamos
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                # Añadir a la lista de exploración
                heapq.heappush(open_list, (f_score[neighbor], neighbor))

    return []  # No se encontró camino
"""

def get_neighbors(grid,node):
  neighbors = []

  x,y = node

  if x > 0:
    neighbors.append((x-1,y))

  if x < grid.shape[1] - 1:
    neighbors.append((x+1,y))

  if y > 0:
    neighbors.append((x,y-1))

  if y < grid.shape[0] -1:
    neighbors.append((x,y+1))

  return neighbors

# Ontology

Correr la ontologia **SOLO UNA VEZ** ,si da problemas usar el comando de borrar y volver a crearla

In [340]:

#Creamos la ontologia
onto = get_ontology("file://ontologia.owl")


with onto:
  class Entity(Thing):
    pass

  class Robot(Entity):
    pass

  class Container(Entity):
    pass

  class Box(Entity):
    pass

  class Place(Thing):
    pass

  class Position(Thing):
    pass

  class has_place(ObjectProperty, FunctionalProperty):
      domain = [Entity]
      range = [Place]

  class has_target(ObjectProperty, FunctionalProperty):
    domain = [Robot]
    range = [Place]

  class has_position(DataProperty, FunctionalProperty):
      domain = [Place]
      range = [str]

  class has_boxes(DataProperty, FunctionalProperty):
      domain = [Container]
      range = [int]

  class has_box(ObjectProperty, FunctionalProperty):
      domain = [Robot]
      range = [Box]

  class is_claimed(DataProperty, FunctionalProperty):
      domain = [Box]
      range = [str]

  class has_capacity(DataProperty, FunctionalProperty):
    domain = [Container]
    range = [int]


'\n#Creamos la ontologia\nonto = get_ontology("file://ontologia.owl")\n\n\nwith onto:\n  class Entity(Thing):\n    pass\n\n  class Robot(Entity):\n    pass\n\n  class Container(Entity):\n    pass\n\n  class Box(Entity):\n    pass\n\n  class Place(Thing):\n    pass\n\n  class Position(Thing):\n    pass\n\n  class has_place(ObjectProperty, FunctionalProperty):\n      domain = [Entity]\n      range = [Place]\n\n  class has_target(ObjectProperty, FunctionalProperty):\n    domain = [Robot]\n    range = [Place]\n\n  class has_position(DataProperty, FunctionalProperty):\n      domain = [Place]\n      range = [str]\n\n  class has_boxes(DataProperty, FunctionalProperty):\n      domain = [Container]\n      range = [int]\n\n  class has_box(ObjectProperty, FunctionalProperty):\n      domain = [Robot]\n      range = [Box]\n\n  class is_claimed(DataProperty, FunctionalProperty):\n      domain = [Box]\n      range = [str]\n\n  class has_capacity(DataProperty, FunctionalProperty):\n    domain = [Conta

In [341]:
#Delete just in case
#onto.destroy(update_relation = True, update_is_a = True)

#Robot Agent

##Robot Agent Base

In [342]:
from logging.config import valid_ident
class RobotAgent(ap.Agent):

  """
    <-- Funcion de Inicializaczion -->
  """
  def setup(self):

    #Agente
    self.agentType = 0

    #Posiciones
    self.has_box = False
    self.target = None

    #Posicionamiento del agente
    self.reserved_position = None #Posicion a reservar antes del siguente paso
    self.original_pos = None #Posicion de inicio

    #Primer paso
    self.first_step = True

    #Contador de pasos
    self.movement_history = 0

#Acciones
    self.actions = (
        self.pick_target,
        self.reserve_to_target,
        self.pick_up_box,
        self.pick_target_container,
        self.drop_box,
        self.return_to_base
    )

#Reglas
    self.rules = (
        self.pick_target_rule,
        self.reserve_to_target_rule,
        self.pick_up_box_rule,
        self.pick_target_container_rule,
        self.drop_box_rule,
        self.return_to_base_rule
    )

  """
    <-- Funcion de Observacion -->
  """

  def see(self, e):
      self.pos = e.positions[self]
      if self.first_step:
          self.reserved_position = self.pos
          self.original_pos = {
              'agent': self,
              'perception': Robot(has_place=Place(has_position=str(e.positions[self])))
          }
          self.first_step = False

      self.box_per = []
      self.container_per = []

      for box in self.model.boxes:
          box_belief = {
              'agent': box,  # Store reference to the actual agent
              'perception': Box(has_place=Place(has_position=str(e.positions[box])), is_claimed=False)
          }
          self.box_per.append(box_belief)

      # Initialize containers
      for container in self.model.containers:
          if container.capacity != 0:
              container_belief = {
                  'agent': container,  # Store reference to the actual agent
                  'perception': Container(has_place=Place(has_position=str(e.positions[container])), has_capacity=container.capacity)
              }
              self.container_per.append(container_belief)


  def next(self):
    for act in self.actions:
      for rule in self.rules:
        if rule(act):
          act()


  def step(self):
    #Ver el entorno
    self.see(self.model.grid)
    self.model.reservations.add_agents([self],[self.pos])
    self.next()

  def update(self):
    pass

  def end(self):
    pass

  """
    <-- Acciones -->
  """

  def pick_target(self):
      chosen_box = None  # Box to choose

      min_distance = float('inf')  # Maximum value to start selection

      # Iterate and find the nearest box
      for box_dict in self.box_per:
          box = box_dict['perception']
          distance = heuristic(self.pos, eval(box.has_place.has_position))  # Distance between the box and the robot

          if distance <= min_distance and box_dict['agent'].has_been_picked == False:
              min_distance = distance
              chosen_box = box_dict

      # No box chosen
      if chosen_box is None:
          return

      chosen_box['perception'].is_claimed = True
      chosen_box['agent'].has_been_picked = True
      self.target = chosen_box  # Assign the actual agent as the target


  """
    Accion de reservar una posicion
  """

  def reserve_to_target(self):

      #Obtenemos los vecinos de nuestra posicion actual
      neighbors = get_neighbors(self.model.grid, self.pos)

      #Variable que obtentra el mejor vecino
      best_neighbor = None

      #Valor maximo para empezar a descartar
      best_f_score = float('inf')

      #Por cada celda en los vecinos
      for neighbor in neighbors:
        #Si el vecino esta vacio
        if neighbor in self.model.reservations.empty:
          #Calculamos el f_score
          tentative_g_score = heuristic(self.pos, neighbor) + 1
          neighbor_f_score = tentative_g_score + heuristic(neighbor, eval(self.target['perception'].has_place.has_position))

          if neighbor_f_score < best_f_score:
              best_f_score = neighbor_f_score
              best_neighbor = neighbor

      #Si obtuvimos un vecino
      if best_neighbor is not None:
        #Nos movemos

        #La logica aqui con el otro grid es redudante, considerar remover
        movement = np.subtract(best_neighbor, self.pos)
        self.model.grid.move_by(self, movement)
        self.model.reservations.remove_agents(self)
        self.reserved_position = best_neighbor
        self.model.reservations.add_agents([self], [self.reserved_position])

        self.movement_history += 1

  """
    Accion de tomar una caja
  """
  def pick_up_box(self):
    self.model.grid.remove_agents(self.target['agent'])
    self.model.boxes.remove(self.target['agent'])
    self.has_box = True
    self.target = None




  """
    Accion de buscar un contenedor
  """

  def pick_target_container(self):
    #El contenedor a escoger
    chosen_container = None

    #Valor maximo del cual empezar a escoger
    min_distance = float('inf')

    #Iteramos por las cajas
    for container in self.container_per:
      #Calculamos distancias
      distance = heuristic(self.pos,eval(container['perception'].has_place.has_position))

      #Si la caja es la mas cercana y no esta llena
      if distance <= min_distance and container['perception'].has_capacity != 0 :
        #La tomamos como candidato
        min_distance = distance
        chosen_container = container

    #Si no encontramos contenedor
    if chosen_container == None:
      return

    #Ligamos el contenedor a un robot
    self.target = chosen_container

  """
    Accion de dejar una caja en el contenedor
  """

  def drop_box(self):
    self.target['perception'].has_capacity -= 1
    self.model.container_capacity -= 1
    self.has_box = False
    self.target = None

  """
    Accion de volver a la base
  """

  def return_to_base(self):
    print("VOLVIENDO")
    self.target = self.original_pos


  """
    <-- Reglas -->
  """


  """
    Regla de escoger un objetivo
  """
  def pick_target_rule(self,act):
    validator = [False,False,False,False]

    if self.target == None:
      validator[0] = True

    if self.has_box == False:
      validator[1] = True

    if len(self.model.boxes) != 0:
      validator[2] = True


    if act == self.pick_target:
      validator[3] = True

    return sum(validator) == 4

  #Regla de mover a objetivo
  def reserve_to_target_rule(self,act):
    validator = [False,False,False]

    if self.target != None:
      validator[0] = True

      if self.pos != eval(self.target['perception'].has_place.has_position):

        validator[1] = True

    if act == self.reserve_to_target:
      validator[2] = True

    return sum(validator) == 3

  def pick_up_box_rule(self,act):
    validator = [False,False,False,False]

    if self.target != None:
      if self.pos == eval(self.target['perception'].has_place.has_position):
        validator[0] = True

    if self.has_box == False:
      validator[1] = True

    if len(self.model.boxes) != 0:
      validator[2] = True

    if act == self.pick_up_box:
      validator[3] = True

    return sum(validator) == 4


  def pick_target_container_rule(self,act):
    validator = [False,False, False]

    if self.target == None:
      validator[0] = True

    if self.has_box == True:
      validator[1] = True

    if act == self.pick_target_container:
      validator[2] = True

    return sum(validator) == 3

  def drop_box_rule(self,act):
    validator = [False,False,False]

    if self.target != None:
      if self.pos == eval(self.target['perception'].has_place.has_position):
        validator[0] = True

    if self.has_box == True:
      validator[1] = True

    if act == self.drop_box:
      validator[2] = True

    return sum(validator) == 3

  def return_to_base_rule(self,act):
    validator = [False,False,False]

    if len(self.model.boxes) == 0:
      validator[0] = True

    if self.has_box == False:
      validator[1] = True

    if act == self.return_to_base:
      validator[2] = True

    return sum(validator) == 3


#Box Agent

In [343]:
class BoxAgent(ap.Agent):

  """
    <-- Funcion de Inicializacion -->
  """
  def setup(self):
    self.agentType = 1
    self.has_been_picked = False
    self.pos = None

  def see(self, e):
    self.pos = e.positions[self]

  def next(self):
    pass

  def step(self):
    self.see(self.model.grid)

  def update(self):
    pass

  def end(self):
    pass


#Container Agent

##Container Agent Base

In [344]:
class ContainerAgent(ap.Agent):

  """
    <-- Funcion de Inicializacion -->
  """
  def setup(self):
    self.agentType = 2
    self.capacity = 100

    pass

  def see(self, e):
    pass

  def next(self):
    pass

  def step(self):
    pass

  def update(self):
    pass

  def end(self):
    pass

#Robot Model

In [345]:
class RobotModel(ap.Model):

  """
    <-- Funcion de Inicializacion -->
  """
  def setup(self):
    self.steps = 0
    self.robots = ap.AgentList(self,self.p.robots,RobotAgent)
    self.boxes = ap.AgentList(self,self.p.boxes,BoxAgent)
    self.containers = ap.AgentList(self,self.p.containers,ContainerAgent)
    self.container_capacity = self.p.containers * 5

    #Instancia Grid
    self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
    self.reservations = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)

    #Asignacion de Agentes
    self.grid.add_agents(self.robots, random=True, empty=True)
    self.grid.add_agents(self.boxes, random=True, empty=True)
    self.grid.add_agents(self.containers, random=True, empty=True)


  def step(self):
    print(f"Step: {self.steps}")
    self.boxes.step()
    self.robots.step()

    #Informacion de la sumulacion
    self.steps += 1
    print(f"Capacity: {self.container_capacity}")
    if self.container_capacity <= 0:
      self.stop()

  def next(self):
    pass

  def update(self):
    self.record('Boxes Delivered', self.container_capacity)

  def end(self):
    robot_id = 0
    for robot in self.robots:
      self.record(f'Robot {robot_id} Movements', robot.movement_history)
      robot_id += 1


#Graphic

#Parameters

Quita le parametro de semilla si da problemas

In [346]:
parameters = {
    'M': 10,
    'N': 10,
    'robots': 5,
    'boxes': 21,
    'containers': 3,
    'steps': 120,
}

#Animation

In [347]:
def animation_plot(model, ax):
    """
    Función de animación
    @param model: modelo
    @param ax: axes (matplotlib)
    """
    # Definición de atributo para tipo de agente
    agent_type_grid = model.grid.attr_grid('agentType')
    # Definición de gráfico con colores (de acuerdo al tipo de agente)
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    # Definición de título del gráfico
    ax.set_title(f"Robot Model \n Time-step: {model.t}, "
                 f"Capacity: {model.container_capacity}")

#Simulation

In [348]:
#SIMULATION:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = RobotModel(parameters)


#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())

Step: 0
Capacity: 15
Step: 1
Capacity: 15
Step: 2
Capacity: 15
Step: 3
Capacity: 15
Step: 4
Capacity: 14
Step: 5
Capacity: 13
Step: 6
Capacity: 13
Step: 7
Capacity: 11
Step: 8
Capacity: 10
Step: 9
Capacity: 10
Step: 10
Capacity: 10
Step: 11
Capacity: 10
Step: 12
Capacity: 9
Step: 13
Capacity: 8
Step: 14
Capacity: 8
Step: 15
Capacity: 6
Step: 16
Capacity: 6
Step: 17
Capacity: 6
Step: 18
Capacity: 5
Step: 19
Capacity: 5
Step: 20
Capacity: 5
Step: 21
Capacity: 4
Step: 22
Capacity: 4
Step: 23
Capacity: 3
Step: 24
Capacity: 3
Step: 25
Capacity: 2
Step: 26
Capacity: 2
Step: 27
Capacity: 1
Step: 28
Capacity: 1
Step: 29
Capacity: 1
Step: 30
Capacity: 1
Step: 31
Capacity: 0


#Plots

##Boxes Plot


Plots de histograma de pasos y cajas con respecto a steps, si se quieren mismos resultados en le mapa y los plots usar una seed.

In [349]:

model = RobotModel(parameters)
results = model.run()

data = results.variables.RobotModel
print(data['Robot 1 Movements'][model.steps])
ax = data['Boxes Delivered'].plot(title='Boxes Delivered', legend=True, xlabel='Steps', ylabel='Boxes Delivered')


"\nmodel = RobotModel(parameters)\nresults = model.run()\n\ndata = results.variables.RobotModel\nprint(data['Robot 1 Movements'][model.steps])\nax = data['Boxes Delivered'].plot(title='Boxes Delivered', legend=True, xlabel='Steps', ylabel='Boxes Delivered')\n"

##Movements Plot

In [350]:

#Plot de los pasos de cada robot

moves_by_robot = []
colors = ['firebrick', 'cornflowerblue', 'green', 'orange', 'indigo']

for i in range(parameters['robots']):
  movements_total = data[f'Robot {i} Movements'][model.steps]
  moves_by_robot.append(movements_total)
  plt.text(i, 2, str(movements_total), ha='center')

plt.bar(range(parameters['robots']), moves_by_robot, color = colors)
plt.xlabel('Robot')
plt.ylabel('Steps')
plt.title('Steps by Robot')
plt.show()


"\n#Plot de los pasos de cada robot\n\nmoves_by_robot = []\ncolors = ['firebrick', 'cornflowerblue', 'green', 'orange', 'indigo']\n\nfor i in range(parameters['robots']):\n  movements_total = data[f'Robot {i} Movements'][model.steps]\n  moves_by_robot.append(movements_total)\n  plt.text(i, 2, str(movements_total), ha='center')\n\nplt.bar(range(parameters['robots']), moves_by_robot, color = colors)\nplt.xlabel('Robot')\nplt.ylabel('Steps')\nplt.title('Steps by Robot')\nplt.show()\n"